In [ ]:
import torch
import torch.nn
import torch.nn.functional as F

import numpy
import matplotlib.pyplot as plt

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 5.4 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 5.2 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.3 MB 5.1 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 747 kB 5.2 MB/s 
     |████████████████████████████████| 370 kB 5.4 MB/s 
     |████████████████████████████████| 482 kB 39.6 MB/s 
     |████████████████████████████████| 41 kB 546 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=9ea02b86f0d590c15c4fc8a860c1e9b838c4383be0d9e45d30d5540bef1541a9
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built to

In [ ]:
import torch_geometric 
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, VGAE, GAE
from torch_geometric.utils import train_test_split_edges 

In [ ]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [ ]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None

In [ ]:
type(dataset)

torch_geometric.datasets.planetoid.Planetoid

In [ ]:
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [ ]:
data
#edge index = adjacency matrix, test_neg_edges -> edges in test set that are not in the graph, test_pos_edge_index -> edges in test set that are in graph, x -> feature matrix, y -> labels

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

In [ ]:
#Define encoder
#in_channels -> number of features 
#out_channels -> 2 * out_channels --> because we have 2 convolutional layers. We go from the input features to double of output features and then in 2nd layer we go from double of output features we go to out features (size of embedding we want to produce)


In [ ]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)


In [ ]:
#Define the autoencoder

In [ ]:
#Parameters
out_channels = 2 #embedding in 2 dimensions
num_features = data.edge_index
epochs = 100

#Model
model = GAE(GCNEncoder(num_features, out_channels))

#Move to GPU, if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

#Initialize the optimizer
optimizer =torch.optim.Adam(model.parameters(), lr = 0.01)


TypeError: ignored

In [ ]:
model

GAE(
  (encoder): GCNEncoder(
    (conv1): GCNConv(3703, 4)
    (conv2): GCNConv(4, 2)
  )
  (decoder): InnerProductDecoder()
)

In [ ]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',


###TENSORBOARD Implementation

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#Parameters
out_channels = 2 #embedding in 2 dimensions
num_features = dataset.num_features
epochs = 100

#Model
model = GAE(GCNEncoder(num_features, out_channels))

#Move to GPU, if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

#Initialize the optimizer
optimizer =torch.optim.Adam(model.parameters(), lr = 0.01)


In [ ]:
writer = SummaryWriter('runs/GAE_experiment'+'2d_100_epochs')

In [ ]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.5964, AP: 0.6346
Epoch: 002, AUC: 0.6045, AP: 0.6515
Epoch: 003, AUC: 0.6116, AP: 0.6567
Epoch: 004, AUC: 0.6156, AP: 0.6610
Epoch: 005, AUC: 0.6180, AP: 0.6639
Epoch: 006, AUC: 0.6204, AP: 0.6662
Epoch: 007, AUC: 0.6212, AP: 0.6681
Epoch: 008, AUC: 0.6219, AP: 0.6703
Epoch: 009, AUC: 0.6242, AP: 0.6730
Epoch: 010, AUC: 0.6254, AP: 0.6750
Epoch: 011, AUC: 0.6258, AP: 0.6761
Epoch: 012, AUC: 0.6264, AP: 0.6780
Epoch: 013, AUC: 0.6266, AP: 0.6801
Epoch: 014, AUC: 0.6264, AP: 0.6822
Epoch: 015, AUC: 0.6257, AP: 0.6843
Epoch: 016, AUC: 0.6250, AP: 0.6861
Epoch: 017, AUC: 0.6238, AP: 0.6874
Epoch: 018, AUC: 0.6230, AP: 0.6890
Epoch: 019, AUC: 0.6223, AP: 0.6900
Epoch: 020, AUC: 0.6219, AP: 0.6908
Epoch: 021, AUC: 0.6216, AP: 0.6915
Epoch: 022, AUC: 0.6213, AP: 0.6917
Epoch: 023, AUC: 0.6219, AP: 0.6925
Epoch: 024, AUC: 0.6223, AP: 0.6929
Epoch: 025, AUC: 0.6231, AP: 0.6937
Epoch: 026, AUC: 0.6234, AP: 0.6942
Epoch: 027, AUC: 0.6241, AP: 0.6948
Epoch: 028, AUC: 0.6254, AP: